In [15]:
import pandas as pd
import requests
from constants import * 

In [16]:
df_weather = pd.read_csv('hly175.csv')
df_bikes = pd.read_csv('data/dublinbike-historical-data-2021-10.csv')

/tmp/ipykernel_210799/1840465739.py:1: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv('hly175.csv')


In [17]:
df_weather

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,16-aug-2003 01:00,0,0,0,9.2,0,8.9,8.5,11.1,95,1021.9
1,16-aug-2003 02:00,0,0,0,9,0,8.7,8.5,11.1,96,1021.7
2,16-aug-2003 03:00,0,0,0,8.2,0,8,7.7,10.5,96,1021.2
3,16-aug-2003 04:00,0,0,0,8.4,0,8.1,7.9,10.7,97,1021.2
4,16-aug-2003 05:00,0,0,0,7.7,0,7.5,7.3,10.2,97,1021.1
...,...,...,...,...,...,...,...,...,...,...,...
179371,31-jan-2024 20:00,0,0.0,0,5.9,0,5.5,5.0,8.7,93,1026.2
179372,31-jan-2024 21:00,0,0.1,0,4.9,0,4.4,3.7,8.0,92,1027.2
179373,31-jan-2024 22:00,0,0.0,0,4.3,0,3.8,3.0,7.6,91,1028.4
179374,31-jan-2024 23:00,0,0.0,0,4.1,0,3.5,2.5,7.3,89,1029.4


### TODO 
# standardize dates to combine weather with historical data
take care with what we use to train (respect to what we can get from the weathe API calls)

In [22]:
df_weather.date = df_weather.date.astype('datetime64[s]')
df_weather.date = pd.to_datetime(df_weather.date).dt.floor('h')
df_weather.date

0        2003-08-16 01:00:00
1        2003-08-16 02:00:00
2        2003-08-16 03:00:00
3        2003-08-16 04:00:00
4        2003-08-16 05:00:00
                 ...        
179371   2024-01-31 20:00:00
179372   2024-01-31 21:00:00
179373   2024-01-31 22:00:00
179374   2024-01-31 23:00:00
179375   2024-02-01 00:00:00
Name: date, Length: 179376, dtype: datetime64[s]

In [24]:
df_bikes.TIME = df_bikes.TIME.astype('datetime64[s]')
df_bikes.TIME = pd.to_datetime(df_bikes.TIME).dt.floor('h')
df_bikes.TIME

0        2021-10-01 00:00:00
1        2021-10-01 00:00:00
2        2021-10-01 00:00:00
3        2021-10-01 00:00:00
4        2021-10-01 00:00:00
                 ...        
163379   2021-10-31 23:00:00
163380   2021-10-31 23:00:00
163381   2021-10-31 23:00:00
163382   2021-10-31 23:00:00
163383   2021-10-31 23:00:00
Name: TIME, Length: 163384, dtype: datetime64[s]

In [28]:
df_bikes

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
0,2,2021-10-01 00:00:00,2021-09-30 23:56:31,BLESSINGTON STREET,20,7,13,OPEN,Blessington Street,53.3568,-6.26814
1,3,2021-10-01 00:00:00,2021-09-30 23:53:57,BOLTON STREET,20,1,19,OPEN,Bolton Street,53.3512,-6.26986
2,4,2021-10-01 00:00:00,2021-09-30 23:56:37,GREEK STREET,20,11,9,OPEN,Greek Street,53.3469,-6.27298
3,5,2021-10-01 00:00:00,2021-09-30 23:54:50,CHARLEMONT PLACE,40,0,40,OPEN,Charlemont Street,53.3307,-6.26018
4,6,2021-10-01 00:00:00,2021-09-30 23:56:04,CHRISTCHURCH PLACE,20,4,16,OPEN,Christchurch Place,53.3434,-6.27012
...,...,...,...,...,...,...,...,...,...,...,...
163379,113,2021-10-31 23:00:00,2021-10-31 23:29:45,MERRION SQUARE SOUTH,40,32,8,OPEN,Merrion Square South,53.3386,-6.24861
163380,114,2021-10-31 23:00:00,2021-10-31 23:28:04,WILTON TERRACE (PARK),40,30,10,OPEN,Wilton Terrace (Park),53.3337,-6.24834
163381,115,2021-10-31 23:00:00,2021-10-31 23:20:42,KILLARNEY STREET,30,8,22,OPEN,Killarney Street,53.3548,-6.24758
163382,116,2021-10-31 23:00:00,2021-10-31 23:21:27,BROADSTONE,30,18,12,OPEN,Broadstone,53.3547,-6.27231


In [29]:
df_weather

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,2003-08-16 01:00:00,0,0,0,9.2,0,8.9,8.5,11.1,95,1021.9
1,2003-08-16 02:00:00,0,0,0,9,0,8.7,8.5,11.1,96,1021.7
2,2003-08-16 03:00:00,0,0,0,8.2,0,8,7.7,10.5,96,1021.2
3,2003-08-16 04:00:00,0,0,0,8.4,0,8.1,7.9,10.7,97,1021.2
4,2003-08-16 05:00:00,0,0,0,7.7,0,7.5,7.3,10.2,97,1021.1
...,...,...,...,...,...,...,...,...,...,...,...
179371,2024-01-31 20:00:00,0,0.0,0,5.9,0,5.5,5.0,8.7,93,1026.2
179372,2024-01-31 21:00:00,0,0.1,0,4.9,0,4.4,3.7,8.0,92,1027.2
179373,2024-01-31 22:00:00,0,0.0,0,4.3,0,3.8,3.0,7.6,91,1028.4
179374,2024-01-31 23:00:00,0,0.0,0,4.1,0,3.5,2.5,7.3,89,1029.4


In [30]:
merged_df = pd.merge(df_bikes, df_weather, left_on='TIME', right_on='date', how='left')

In [31]:
merged_df

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,...,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,2,2021-10-01 00:00:00,2021-09-30 23:56:31,BLESSINGTON STREET,20,7,13,OPEN,Blessington Street,53.3568,...,0,0.0,0,13.9,0,12.1,10.4,12.6,79,1002.1
1,3,2021-10-01 00:00:00,2021-09-30 23:53:57,BOLTON STREET,20,1,19,OPEN,Bolton Street,53.3512,...,0,0.0,0,13.9,0,12.1,10.4,12.6,79,1002.1
2,4,2021-10-01 00:00:00,2021-09-30 23:56:37,GREEK STREET,20,11,9,OPEN,Greek Street,53.3469,...,0,0.0,0,13.9,0,12.1,10.4,12.6,79,1002.1
3,5,2021-10-01 00:00:00,2021-09-30 23:54:50,CHARLEMONT PLACE,40,0,40,OPEN,Charlemont Street,53.3307,...,0,0.0,0,13.9,0,12.1,10.4,12.6,79,1002.1
4,6,2021-10-01 00:00:00,2021-09-30 23:56:04,CHRISTCHURCH PLACE,20,4,16,OPEN,Christchurch Place,53.3434,...,0,0.0,0,13.9,0,12.1,10.4,12.6,79,1002.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163379,113,2021-10-31 23:00:00,2021-10-31 23:29:45,MERRION SQUARE SOUTH,40,32,8,OPEN,Merrion Square South,53.3386,...,0,0.0,0,8.4,0,7.9,7.3,10.2,92,983.4
163380,114,2021-10-31 23:00:00,2021-10-31 23:28:04,WILTON TERRACE (PARK),40,30,10,OPEN,Wilton Terrace (Park),53.3337,...,0,0.0,0,8.4,0,7.9,7.3,10.2,92,983.4
163381,115,2021-10-31 23:00:00,2021-10-31 23:20:42,KILLARNEY STREET,30,8,22,OPEN,Killarney Street,53.3548,...,0,0.0,0,8.4,0,7.9,7.3,10.2,92,983.4
163382,116,2021-10-31 23:00:00,2021-10-31 23:21:27,BROADSTONE,30,18,12,OPEN,Broadstone,53.3547,...,0,0.0,0,8.4,0,7.9,7.3,10.2,92,983.4


In [32]:
# Specify the timestamp you want to filter for
timestamp_to_find = '2021-10-31 23:00:00'

# Using .loc[]
result = merged_df.loc[merged_df['TIME'] == timestamp_to_find]

print(result)


        STATION ID                TIME         LAST UPDATED  \
163164           2 2021-10-31 23:00:00  2021-10-31 22:57:42   
163165           3 2021-10-31 23:00:00  2021-10-31 22:55:41   
163166           4 2021-10-31 23:00:00  2021-10-31 22:54:01   
163167           5 2021-10-31 23:00:00  2021-10-31 22:59:38   
163168           6 2021-10-31 23:00:00  2021-10-31 22:56:13   
...            ...                 ...                  ...   
163379         113 2021-10-31 23:00:00  2021-10-31 23:29:45   
163380         114 2021-10-31 23:00:00  2021-10-31 23:28:04   
163381         115 2021-10-31 23:00:00  2021-10-31 23:20:42   
163382         116 2021-10-31 23:00:00  2021-10-31 23:21:27   
163383         117 2021-10-31 23:00:00  2021-10-31 23:23:39   

                         NAME  BIKE_STANDS  AVAILABLE_BIKE_STANDS  \
163164     BLESSINGTON STREET           20                     10   
163165          BOLTON STREET           20                      8   
163166           GREEK STREET       

In [34]:
result.iloc[0]

STATION ID                                 2
TIME                     2021-10-31 23:00:00
LAST UPDATED             2021-10-31 22:57:42
NAME                      BLESSINGTON STREET
BIKE_STANDS                               20
AVAILABLE_BIKE_STANDS                     10
AVAILABLE_BIKES                           10
STATUS                                  OPEN
ADDRESS                   Blessington Street
LATITUDE                             53.3568
LONGITUDE                           -6.26814
date                     2021-10-31 23:00:00
ind                                        0
rain                                     0.0
ind.1                                      0
temp                                     8.4
ind.2                                      0
wetb                                     7.9
dewpt                                    7.3
vappr                                   10.2
rhum                                      92
msl                                    983.4
Name: 1631

In [35]:
# Specify the timestamp you want to filter for
timestamp_to_find = '2021-10-31 23:00:00'

# Using .loc[]
result = df_weather.loc[df_weather.date == timestamp_to_find]

print(result)

                      date  ind rain  ind.1 temp  ind.2 wetb dewpt vappr rhum  \
159646 2021-10-31 23:00:00    0  0.0      0  8.4      0  7.9   7.3  10.2   92   

          msl  
159646  983.4  


Perfect, the merge is looking up and finding the rate weather info in the dataset